In [1]:
import requests
import json
from csv import writer
import pandas as pd

with open("count.txt", "r") as f:
    rowNum = int(f.read())

count = rowNum + 1

#GLOBAL VARIABLES
uniswapV2 = "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"
uniswapV3 = "0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45"

# change file here
addressLists = pd.read_csv("tokens.csv")
walletsAndVersions = {}

for i, row in addressLists[rowNum:].iterrows():
    # while count < 2:
    token = row["TokenPair"]

    variables = {"input": token}
    query = """
            query($input: String!){
                swaps(
                    where: {
                        pair_: {id: $input},
                        timestamp_gte: 1672880171,
                        timestamp_lte: 1672881911
                    }
                    orderDirection: asc
                    orderBy: timestamp
                    first: 1000
                ) {
                    sender
                    to
                }
            }
        """

    resp = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
                           json={'query': query, 'variables': variables})

    json_data = json.loads(resp.text)
    
    json_data = json_data["data"]["swaps"]

    for eachData in json_data:
        if(eachData["sender"] == uniswapV2 and eachData["to"] != uniswapV2):
            walletsAndVersions[eachData["to"]] = 2
        
        elif (eachData["sender"] == uniswapV3 and eachData["to"] != uniswapV3):
            walletsAndVersions[eachData["to"]] = 3

    count += 1

    with open("count.txt", "w") as f:
        f.write(str(count-1))

with open('wallets.csv', 'a') as fd:
    for key in walletsAndVersions:
        fd.write(key + "," + str(walletsAndVersions[key]) + "\n")

In [2]:
wallets = pd.read_csv("wallets.csv")
wallets = wallets["Wallet"]
wallets = wallets.drop_duplicates()

weiDivisible = 1000000000000000000

for user in wallets:
    url = f"https://api.etherscan.io/api?module=account&action=balance&address={user}&tag=latest&apikey=7YRRHUFSDSM5BXCIM8FY3NPJ192WYKRC1G"
    response = requests.get(url)
    jsonData = json.loads(response.text)
    data = float(jsonData["result"])
    data = data / weiDivisible
    

    with open('balance.csv', 'a') as fd:
        fd.write(f"{user}, {data}\n")

KeyError: 'Balance'

In [3]:
pd.set_option('display.max_rows', None)
userInfo = pd.read_csv("balance.csv")
userInfo.sort_values('Balance', ascending=False)

,Wallet,Balance
60,0x71a1718cf9aa881511bba3321ef25382d92fc64e,9.170759
12,0x9834d13b34d91a68ec912b58c0f84c1745cd42cf,6.344307
10,0x93d9c3679b40732b2660136a5547d4d44437e627,5.268873
33,0xb32f47299c268bfcd610d84d26c7a5c64bb36e4f,3.078252
45,0x39ada6995cd989c15024053b0ce14c8bd9627a84,2.853488
11,0x8873fecba07135a4ce06f9d48a9c89f33ab883f4,2.531500
20,0x9973af61dabcc7ced6095ecf4bffc0f0a2aeeed4,2.499501
69,0x269fbf058682d878e733866cdbb15ab510cf4c1a,2.394731
4,0xe2dcbf39dc468edd257fe09aa3b1e46f15222039,2.384282
28,0x82282071424d3c1eaf2f0a3630fa396862a2eba5,2.297785
